This function computes the interpolant for the instantaneous momentum barrier equation:

\begin{equation}
 \mathbf{x}'=\nu \rho \mathbf{J} \mathbf{\nabla \hat{\omega} (\mathbf{x}, t)}, \label{eq: linearmomentum} \tag{1}
\end{equation}
 
 where $ \nu $ is the viscosity, $ \rho $ the density of the fluid, $ \mathbf{J} = \begin{pmatrix} 0 && 1 \\ -1 && 0 \end{pmatrix} $ and $ \hat{\omega} $ is the scalar valued vorticity in 2D.
 
For simplicity we can normalize equation $ \ref{eq: linearmomentum} $ by $ \nu $ and $ \rho $ and thus simply omit them for simplicity as they will play no role when exploring the structures of the field $ \mathbf{x'} $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx)| X-meshgrid with Ny=Nx=1024 over which $ \hat{\omega}(\mathbf{x}, t) $ is defined |
| Y | array (Ny, Nx)| Y-meshgrid with Ny=Nx=1024 over which $ \hat{\omega}(\mathbf{x}, t) $ is defined |
| omega | array (Ny, Nx, N) | $ \hat{\omega}(\mathbf{x}, t) $ with $ N = 251 $ being the resolution in time |
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| Interpolant_Jgrad_omega | list (2,) | Interpolant_Jgrad_omega[0]: Interpolant for x-component of $ \mathbf{x}' $ <br /> Interpolant_Jgrad_omega[1]: Interpolant for y-component of $ \mathbf{x}' $ |

In [ ]:
# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# Import numpy 
import numpy as np

In [3]:
def InstantaneousActiveMomentum(X, Y, omega, aux_grid):
    '''
    Compute interpolant of right-hand-side of Eulerian momentum barrier equation.
    
    Parameters:
        X:              array (NY, NX), X-meshgrid of data domain
        Y:              array (NY, NX), Y-meshgrid of data domain
        omega:          array (NY, NX), vorticity over meshgrid
        aux_grid:       list(2,), specifies spacing of x/y auxiliary grid components
        
    Returns:
        Interpolant_Jgrad_omega: list(2,) of Interpolant-objects for right-hand-side of Eulerian momentum barrier equation.
        Interpolant_Jgrad_omega[0] --> Interpolant for x-component
        Interpolant_Jgrad_omega[1] --> Interpolant for y-component
    '''
    
    # auxiliary grid
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    # compute interpolant for omega over meshgrid X, Y    
    Interpolant_omega = RBS(Y[:,0], X[0,:], omega)
    
    Jgrad_omega = np.zeros((X.shape[0], X.shape[1], 2))
    
    # compute x'=J grad(omega), with J = [[0, 1], [-1, 0]]
    for i in range(X.shape[0]):
        
        for j in range(Y.shape[1]):
            
            # apply periodic boundary conditions to point 'x'
            x = np.array([X[i, j], Y[i, j]])
            
            # evaluate grad(omega) at 'x' using auxiliary grid
            xR = x[0] + rho_x
            OmegaR = Interpolant_omega(x[1], xR)[0][0]
            
            xL = x[0] - rho_x
            OmegaL = Interpolant_omega(x[1], xL)[0][0]
            
            xU = x[1] + rho_y
            OmegaU = Interpolant_omega(xU, x[0])[0][0]
            
            xD = x[1] - rho_y
            OmegaD = Interpolant_omega(xD, x[0])[0][0]
            
            Jgrad_omega[i, j, 0] = -(OmegaU - OmegaD)/(2*rho_y)
            Jgrad_omega[i, j, 1] = (OmegaR - OmegaL)/(2*rho_x)

    Interpolant_Jgrad_omega = []
    Interpolant_Jgrad_omega.append(RBS(Y[:,0], X[0,:], Jgrad_omega[:,:,0]))
    Interpolant_Jgrad_omega.append(RBS(Y[:,0], X[0,:], Jgrad_omega[:,:,1]))
    
    return Interpolant_Jgrad_omega